# Applying Indicators to Price Data
Takes the output from `iq_feed_cleaning.ipynb` and for each row, adds indicators. \
This is preferred over calculating the indicators for each timestamp/date, as it saves 
a lot of computations. \
It does require us to do a look-up, but so does the alternative.

In [ ]:
import os
import pandas as pd
from src.config import config
from src.utils.tickers import get_tickers
import concurrent.futures

import tqdm as tqdm

In [ ]:
tickers = get_tickers(config.data.iqfeed.daily.cleaned)

---------

In [ ]:
from arch import arch_model
def get_insample_conditional_volatilies(ts: pd.Series):
    model = (arch_model(ts * 100, 
                        mean = 'Constant', 
                        vol = 'GARCH', 
                        p = 1, o = 0, q = 1)
                .fit(update_freq = 0, disp = 'off'))

    # coefs = model.params
    cond_vol = model.conditional_volatility
    # Downscale after having upscaled for numerical stability 
    cond_vol /= 100
    return cond_vol

-----------

In [ ]:
def add_indicators(ticker):
    prices = pd.read_parquet(path=f"{config.data.iqfeed.daily.cleaned}/{ticker}_daily.parquet")
    
    # TODO: train_test split... prevent forward looking bias!
    # Requires train test split before using this module...  
    prices["std_252"] = prices["adj_close"].pct_change().rolling(252, min_periods=252).std()*(252**0.5)
    prices["r"] = prices["adj_close"] / prices["adj_close"].shift() - 1 
    prices["cond_vola"] = get_insample_conditional_volatilies(prices["r"].dropna())
    prices["dollar_volume"] = prices["adj_volume"] * (prices["adj_close"] + prices["adj_open"])/2
    prices["r_intra_(t-1)"] = (prices["adj_close"] / prices["adj_open"] - 1).shift(periods=1)
    prices["unadj_open"] = prices["adj_open"] / prices["cum_split_ratio"]
    
    prices.to_parquet(path=f"{config.data.iqfeed.daily.cleaned}/{ticker}_daily.parquet")

In [ ]:
# pool_obj = ThreadPoolExecutor(max_workers=os.cpu_count()-1)
# ans = pool_obj.map(add_indicators, tickers)
# result = list(ans)

In [ ]:
with tqdm(total=len(tickers), desc="tickers (indicator_applicator)") as pbar:
    # let's give it some more threads:
    with concurrent.futures.ThreadPoolExecutor(max_workers=os.cpu_count()) as executor:
        futures = {executor.submit(add_indicators, ticker): ticker for ticker in tickers}
        for future in concurrent.futures.as_completed(futures):
            ticker = futures[future]
            pbar.update(1)